# **Checkpoint** **save**

In [ ]:
import tensorflow as tf
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

# **Model Setup and Data Exploration**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from os import listdir
from os.path import isdir, join

def count_folders_and_images(directory_path):
    folders_count = 0
    images_per_folder = {}

    for folder_name in listdir(directory_path):
        folder_path = join(directory_path, folder_name)
        if isdir(folder_path):
            folders_count += 1
            images_count = len(listdir(folder_path))
            images_per_folder[folder_name] = images_count

    return folders_count, images_per_folder

# Load and preprocess the data
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_directory = './path_to_new_train_directory'
valid_directory = './path_to_new_valid_directory'

training_set = train_datagen.flow_from_directory(train_directory, target_size=(64, 64), batch_size=32, class_mode='sparse', color_mode='grayscale')
test_set = test_datagen.flow_from_directory(valid_directory, target_size=(64, 64), batch_size=32, class_mode='sparse', color_mode='grayscale')

# Count folders and images in train and valid directories
train_folders_count, train_images_per_folder = count_folders_and_images(train_directory)
valid_folders_count, valid_images_per_folder = count_folders_and_images(valid_directory)

print("Number of folders in train directory:", train_folders_count)
print("Number of images per folder in train directory:")
for folder_name, images_count in train_images_per_folder.items():
    print(f"{folder_name}: {images_count} images")

print("\nNumber of folders in valid directory:", valid_folders_count)
print("Number of images per folder in valid directory:")
for folder_name, images_count in valid_images_per_folder.items():
    print(f"{folder_name}: {images_count} images")


Found 5998 images belonging to 10 classes.
Found 1260 images belonging to 10 classes.
Number of folders in train directory: 10
Number of images per folder in train directory:
0: 600 images
1: 600 images
2: 600 images
3: 600 images
4: 600 images
5: 600 images
6: 600 images
7: 600 images
8: 600 images
9: 598 images

Number of folders in valid directory: 10
Number of images per folder in valid directory:
0: 126 images
1: 126 images
2: 126 images
3: 126 images
4: 126 images
5: 126 images
6: 126 images
7: 126 images
8: 126 images
9: 126 images


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.utils import get_custom_objects

# Define a learning rate schedule
def learning_rate_schedule(epoch, lr):
    if epoch < 20:
        return lr
    elif epoch < 40:
        return lr * 0.5
    else:
        return lr * 0.1

# Define the Monte Carlo Dropout layer
class MonteCarloDropout(Dropout):
    def dropped_inputs(self, inputs, training=None):
        return super().call(inputs, training=training)

# Register the custom layer
get_custom_objects()['MonteCarloDropout'] = MonteCarloDropout

weight_decay = 1e-5

model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(64, 64, 1)))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(MonteCarloDropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(64, 64, 1)))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(64, 64, 1)))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(MonteCarloDropout(0.3))


model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MonteCarloDropout(0.4))

model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.6))

model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.6))

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(BatchNormalization())
model.add(Dense(512, activation="relu"))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))


# Use the learning rate scheduler with the Adam optimizer
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define the learning rate adjustment callback
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)

# Use the learning rate scheduler callback and learning rate adjustment callback during training
callbacks = [LearningRateScheduler(learning_rate_schedule), lr_reducer]

# Train the model
epochs = 200
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    history = model.fit(training_set,
                        epochs=1,
                        steps_per_epoch=len(training_set),
                        validation_data=test_set,
                        validation_steps=len(test_set),
                        callbacks=callbacks,
                        verbose=1)

    # Access the accuracy and loss for each epoch
    accuracy = history.history['accuracy'][0]
    loss = history.history['loss'][0]
    val_accuracy = history.history['val_accuracy'][0]
    val_loss = history.history['val_loss'][0]

    print(f"Accuracy: {accuracy:.4f} | Loss: {loss:.4f} | Val Accuracy: {val_accuracy:.4f} | Val Loss: {val_loss:.4f}")

# Save the trained model
model.save('modelweightbalanceddata.h5')


Epoch 1/200
188/188 [==============================] - 168s 863ms/step - loss: 2.7750 - accuracy: 0.1039 - val_loss: 2.3865 - val_accuracy: 0.1040 - lr: 0.0010
Accuracy: 0.1039 | Loss: 2.7750 | Val Accuracy: 0.1040 | Val Loss: 2.3865
Epoch 2/200
188/188 [==============================] - 161s 856ms/step - loss: 2.5178 - accuracy: 0.1087 - val_loss: 2.4365 - val_accuracy: 0.1198 - lr: 0.0010
Accuracy: 0.1087 | Loss: 2.5178 | Val Accuracy: 0.1198 | Val Loss: 2.4365
Epoch 3/200
188/188 [==============================] - 172s 916ms/step - loss: 2.4174 - accuracy: 0.1149 - val_loss: 2.3939 - val_accuracy: 0.1365 - lr: 0.0010
Accuracy: 0.1149 | Loss: 2.4174 | Val Accuracy: 0.1365 | Val Loss: 2.3939
Epoch 4/200
188/188 [==============================] - 185s 985ms/step - loss: 2.3954 - accuracy: 0.1150 - val_loss: 2.3874 - val_accuracy: 0.1135 - lr: 0.0010
Accuracy: 0.1150 | Loss: 2.3954 | Val Accuracy: 0.1135 | Val Loss: 2.3874
Epoch 5/200
188/188 [==============================] - 171s 908m

188/188 [==============================] - 185s 983ms/step - loss: 0.9326 - accuracy: 0.6442 - val_loss: 0.5988 - val_accuracy: 0.7389 - lr: 0.0010
Accuracy: 0.6442 | Loss: 0.9326 | Val Accuracy: 0.7389 | Val Loss: 0.5988
Epoch 37/200
188/188 [==============================] - 186s 987ms/step - loss: 0.9356 - accuracy: 0.6439 - val_loss: 0.5681 - val_accuracy: 0.7746 - lr: 0.0010
Accuracy: 0.6439 | Loss: 0.9356 | Val Accuracy: 0.7746 | Val Loss: 0.5681
Epoch 38/200
188/188 [==============================] - 177s 942ms/step - loss: 0.8886 - accuracy: 0.6687 - val_loss: 0.5949 - val_accuracy: 0.7540 - lr: 0.0010
Accuracy: 0.6687 | Loss: 0.8886 | Val Accuracy: 0.7540 | Val Loss: 0.5949
Epoch 39/200
188/188 [==============================] - 169s 900ms/step - loss: 0.9051 - accuracy: 0.6642 - val_loss: 0.6080 - val_accuracy: 0.7365 - lr: 0.0010
Accuracy: 0.6642 | Loss: 0.9051 | Val Accuracy: 0.7365 | Val Loss: 0.6080
Epoch 40/200
188/188 [==============================] - 170s 906ms/step -

188/188 [==============================] - 166s 884ms/step - loss: 0.6387 - accuracy: 0.7709 - val_loss: 0.5182 - val_accuracy: 0.8103 - lr: 0.0010
Accuracy: 0.7709 | Loss: 0.6387 | Val Accuracy: 0.8103 | Val Loss: 0.5182
Epoch 72/200
188/188 [==============================] - 166s 884ms/step - loss: 0.6369 - accuracy: 0.7631 - val_loss: 0.4320 - val_accuracy: 0.8119 - lr: 0.0010
Accuracy: 0.7631 | Loss: 0.6369 | Val Accuracy: 0.8119 | Val Loss: 0.4320
Epoch 73/200
188/188 [==============================] - 166s 884ms/step - loss: 0.6521 - accuracy: 0.7583 - val_loss: 0.3892 - val_accuracy: 0.8651 - lr: 0.0010
Accuracy: 0.7583 | Loss: 0.6521 | Val Accuracy: 0.8651 | Val Loss: 0.3892
Epoch 74/200
188/188 [==============================] - 168s 891ms/step - loss: 0.6169 - accuracy: 0.7818 - val_loss: 0.3836 - val_accuracy: 0.8595 - lr: 0.0010
Accuracy: 0.7818 | Loss: 0.6169 | Val Accuracy: 0.8595 | Val Loss: 0.3836
Epoch 75/200
188/188 [==============================] - 165s 879ms/step -

188/188 [==============================] - 197s 1s/step - loss: 0.5297 - accuracy: 0.8540 - val_loss: 0.3387 - val_accuracy: 0.9262 - lr: 0.0010
Accuracy: 0.8540 | Loss: 0.5297 | Val Accuracy: 0.9262 | Val Loss: 0.3387
Epoch 142/200
188/188 [==============================] - 190s 1s/step - loss: 0.5119 - accuracy: 0.8633 - val_loss: 0.3318 - val_accuracy: 0.9302 - lr: 0.0010
Accuracy: 0.8633 | Loss: 0.5119 | Val Accuracy: 0.9302 | Val Loss: 0.3318
Epoch 143/200
188/188 [==============================] - 187s 997ms/step - loss: 0.5005 - accuracy: 0.8666 - val_loss: 0.3217 - val_accuracy: 0.9222 - lr: 0.0010
Accuracy: 0.8666 | Loss: 0.5005 | Val Accuracy: 0.9222 | Val Loss: 0.3217
Epoch 144/200
188/188 [==============================] - 200s 1s/step - loss: 0.4925 - accuracy: 0.8661 - val_loss: 0.3653 - val_accuracy: 0.9056 - lr: 0.0010
Accuracy: 0.8661 | Loss: 0.4925 | Val Accuracy: 0.9056 | Val Loss: 0.3653
Epoch 145/200
188/188 [==============================] - 207s 1s/step - loss: 0

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
from tensorflow.keras.models import load_model


In [ ]:
# Make predictions on the test set
predictions = model.predict(test_set)

# Convert the predictions into class labels
predicted_labels = np.argmax(predictions, axis=1)

# Get the true labels from the test set
true_labels = test_set.labels

# Calculate the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)
